In [1]:
import torch
import numpy as np
import pandas as pd
from diffroute import get_node_idxs

from src.data import Dataset
from src.model import init_model
from src.helpers import training_loop, hydro_metrics, extract_train, extract_test

### Experiment

In [2]:
N = 10
dataset = "40_P"
device = "cuda:6"
irf_fns = [ "hayami", 'pure_lag', 'linear_storage', 'nash_cascade', 'muskingum']

n_iter =40
n_epoch=400
lr = .001
wd = .002
init_window = 100  # LSTM init window

ds = Dataset.load_from_folder(dataset, device)

### Lumped model training and evaluation

We will reuse the trained lumped models for the two-step approach evaluation in the next cell

In [ ]:
lumped_res = []
for i in range(N):
    # Init model and optimizer
    model = init_model(ds.g, inp_size=ds.X.shape[-1], 
                            dt = .25, irf_fn="hayami", # Does not matter, don't use 
                            device=device)
    model.lumped = True
    opt = torch.optim.Adam([
        {'params': model.runoff_model.parameters(), 'lr': lr, 'weight_decay': wd},
    ])
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=100, gamma=0.3)

    # Train
    losses = training_loop(ds, model, opt, n_iter=n_iter, 
                           n_epoch=n_epoch, 
                           scheduler=scheduler,
                           clip_grad_norm=1.0)

    # Extract and record results
    ytr, otr = extract_train(model, ds, init_window)
    yte, ote = extract_test(model, ds, init_window)
    lumped_res.append((model.cpu(), ytr, otr, yte, ote))

 60%|██████    | 24/40 [00:00<00:00, 76.10it/s]

### Plot metrics

In [ ]:
yte = [x[3] for x in lumped_res]
ote = [x[4] for x in lumped_res]
lumped_results = pd.DataFrame([hydro_metrics(out, y) \
                        for out,y in zip(ote, yte)])

In [19]:
pd.DataFrame({"mean":lumped_results.mean(), "std":lumped_results.std()}),

(                  mean       std
 NSE           0.896146  0.003794
 KGE_2009      0.751330  0.013832
 KGE_2012      0.794849  0.014489
 r             0.955176  0.002388
 alpha         0.849656  0.009785
 gamma         1.052687  0.014171
 beta          0.807241  0.012859
 MAE           0.145467  0.002591
 MSE           0.178399  0.006517
 n         34940.000000  0.000000,)

### Two-step: default parameter ranges -> converge to low NSE

In [ ]:
# Routing model converges fast because linear
n_iter = 10 
n_epoch = 20

In [ ]:
two_step_res = {}

for irf_fn in irf_fns:
    print(irf_fn)
    
    two_step_res[irf_fn]=[]
    for i,r in enumerate(lumped_res):
        runoff = r[0].runoff_model.to(device)
        
        model = init_model(ds.g, inp_size=ds.X.shape[-1], 
                            dt = .25, irf_fn=irf_fn, 
                            device=device)
        model.runoff_model = runoff
        opt = torch.optim.Adam([
            {'params': [model.routing_params], 'lr': 1e-1, 'weight_decay': 0.0}
        ])
        losses = training_loop(ds, model, opt, 
                               n_iter=n_iter, 
                               n_epoch=n_epoch) 
        ytr, otr = extract_train(model, ds, init_window)
        yte, ote = extract_test(model, ds, init_window)

        two_step_res[irf_fn].append((model.cpu(), ytr, otr, yte, ote))

In [ ]:
yte = {k: [x[3] for x in v] for k,v in two_step_res.items()}
ote = {k: [x[4] for x in v] for k,v in two_step_res.items()}
results = {k:pd.DataFrame([hydro_metrics(out, y) \
                           for out,y in zip(ote[k], yte[k])])
           for k in ote}

In [20]:
{k:v.std() for k,v in results.items()}

{'hayami': NSE         0.002302
 KGE_2009    0.010358
 KGE_2012    0.009698
 r           0.002012
 alpha       0.007057
 gamma       0.014106
 beta        0.011698
 MAE         0.002556
 MSE         0.003954
 n           0.000000
 dtype: float64,
 'pure_lag': NSE         0.003700
 KGE_2009    0.011679
 KGE_2012    0.011708
 r           0.001922
 alpha       0.008663
 gamma       0.014650
 beta        0.011721
 MAE         0.002781
 MSE         0.006355
 n           0.000000
 dtype: float64,
 'linear_storage': NSE         0.003529
 KGE_2009    0.011548
 KGE_2012    0.011723
 r           0.001779
 alpha       0.008910
 gamma       0.015311
 beta        0.011698
 MAE         0.002810
 MSE         0.006062
 n           0.000000
 dtype: float64,
 'nash_cascade': NSE         0.003464
 KGE_2009    0.011618
 KGE_2012    0.011780
 r           0.001747
 alpha       0.008849
 gamma       0.015105
 beta        0.011709
 MAE         0.002805
 MSE         0.005951
 n           0.000000
 dtype: float

In [21]:
{k:v.mean() for k,v in results.items()}

{'hayami': NSE             0.883342
 KGE_2009        0.713005
 KGE_2012        0.802649
 r               0.957214
 alpha           0.789907
 gamma           0.976044
 beta            0.809418
 MAE             0.143680
 MSE             0.200393
 n           34940.000000
 dtype: float64,
 'pure_lag': NSE             0.893247
 KGE_2009        0.734096
 KGE_2012        0.803889
 r               0.957734
 alpha           0.819632
 gamma           1.012736
 beta            0.809437
 MAE             0.144179
 MSE             0.183378
 n           34940.000000
 dtype: float64,
 'linear_storage': NSE             0.893322
 KGE_2009        0.734636
 KGE_2012        0.803732
 r               0.957639
 alpha           0.820490
 gamma           1.013828
 beta            0.809416
 MAE             0.144418
 MSE             0.183250
 n           34940.000000
 dtype: float64,
 'nash_cascade': NSE             0.893084
 KGE_2009        0.734764
 KGE_2012        0.803692
 r               0.957456
 alpha   

### Two-step: Permissive parameter ranges

This setting converges to high NSE with instantaneous IRFs.

These are the results presented in the paper

In [ ]:
from src.model import PARAMS_BOUNDS

PARAMS_BOUNDS["hayami"][1][:,-1]=20
PARAMS_BOUNDS['linear_storage'][0][:]=.01
PARAMS_BOUNDS['linear_storage'][-1][:]=-3
PARAMS_BOUNDS['nash_cascade'][0][:]=.005
PARAMS_BOUNDS['muskingum'][1][0,0]=1

In [ ]:
two_step_res = {}

for irf_fn in irf_fns:
    print(irf_fn)
    
    two_step_res[irf_fn]=[]
    for i,r in enumerate(lumped_res):
        runoff = r[0].runoff_model.to(device)
        
        model = init_model(ds.g, inp_size=ds.X.shape[-1], 
                            dt = .25, irf_fn=irf_fn, 
                            device=device)
        model.runoff_model = runoff
        opt = torch.optim.Adam([
            {'params': [model.routing_params], 'lr': 1e-1, 'weight_decay': 0.0}
        ])
        losses = training_loop(ds, model, opt, 
                               n_iter=n_iter, 
                               n_epoch=n_epoch) 
        ytr, otr = extract_train(model, ds, init_window)
        yte, ote = extract_test(model, ds, init_window)

        two_step_res[irf_fn].append((model.cpu(), ytr, otr, yte, ote))

In [ ]:
yte = {k: [x[3] for x in v] for k,v in two_step_res.items()}
ote = {k: [x[4] for x in v] for k,v in two_step_res.items()}
results = {k:pd.DataFrame([hydro_metrics(out, y) \
                           for out,y in zip(ote[k], yte[k])])
           for k in ote}

In [ ]:
{k:v.std() for k,v in results.items()}

In [ ]:
{k:v.mean() for k,v in results.items()}